In [ ]:
import dedalus.public as de
import numpy as np
import matplotlib.pyplot as plt
from dedalus.tools import post
import file_tools as flt
import glob
import pandas as pd
from scipy import ndimage as ndi
from skfmm import distance

import interpolation as ip
import matplotlib as mpl
mpl.rcParams['pcolor.shading'] = 'gouraud'

colors = {1: 'C4', 2: 'C1', 3: 'lightgray'}
from matplotlib.colors import to_rgb
colors = {i: np.array(to_rgb(colors[i])) for i in colors}

# 2D analysis functions

In [ ]:
def mag(x): 
    return np.log10(np.abs(x)+1e-16)

def concat_lists(lists):
    out = []
    for l in lists:
        out.extend(l)
    return out

def list_saves(files):
    if isinstance(files[0],str): files = [files]
    sorted_files = [f for series in files for f in sorted(series, key=lambda s: int(s.split('_s')[1].split('.')[0]))]
    saves = []
    for i, f in enumerate(sorted_files):
        t, it = flt.load_data(f, 'sim_time','iteration',group='scales',asscalar=False)
        for j, (tj, itj) in enumerate(zip(t,it)):
            if not (i != 0 and itj == 0): # don't double count start of restarts
                saves.append({
                    'file_index':i, 
                    'file':f, 
                    'save_index':j, 
                    'sim_time':tj, 
                    'sim_it':itj})
    for i, save in enumerate(saves):
        save['index'] = i
    return saves


from scipy.interpolate import RectBivariateSpline
def uniform_interpolate_chebyshev(arr,grid_old,grid_new):
    return RectBivariateSpline(*grid_old, arr)(*grid_new,grid=True)

def recentre_drop(grids_old,c1,c2):
    grids_new = [np.linspace(-1,1,len(grids_old[0])) for grid in grids_old]
    c3 = 1-c1-c2
    c1,c2,c3 = [uniform_interpolate_chebyshev(arr,grids_old,grids_new) for arr in [c1,c2,c3]]
    dist = distance(c3-.5,dx=float(grids_new[0][1]-grids_new[0][0]),periodic=True)
    minimum_ind_x = np.unravel_index(np.argmin(dist,axis=None),dist.shape)[0]
    centre = len(grids_old[0])//2
    shift = centre - minimum_ind_x
    return np.roll(c1,shift,axis=0), np.roll(c2,shift,axis=0)

def clip(c):
    copy = c.copy()
    copy[c<0] = 0
    copy[c>1] = 1
    return copy

def color_func(c1,c2):
    sel1 = np.s_[...,None]
    sel2 = np.s_[None,None,...]
    return clip(colors[1][sel2]*c1[sel1] + colors[2][sel2]*c2[sel1] + colors[3][sel2]*(1-c1-c2)[sel1])

def plot_concentrations(c1,c2,ax):
    color_arr = color_func(c1,c2)
    ax.imshow(color_arr.transpose(1,0,2),origin='lower',extent=[-1,1,-1,1])
    ax.set(aspect=1,xticks=[],yticks=[])
    for spine in ax.spines: ax.spines[spine].set_visible(False)
    return fig, ax

def argmax_nd(arr):
    return np.unravel_index(np.argmax(arr, axis=None), arr.shape)

def filter_saves(saves):
    for i, save in enumerate(saves):
        if i > 0:
            t0, t1 = saves[i-1]['sim_time'], saves[i]['sim_time']
            if t1 < t0-1e-4:
                for j in range(i,len(saves)):
                    saves[j]['sim_time'] += t0-t1
    saves = [save for i, save in enumerate(saves) if save['sim_time'] > saves[i-1]['sim_time']]
    return saves


def label_2D(arr):
    N = len(arr)
    labels2D = [ndi.label(arr[i],structure=np.ones((3,3))) for i in range(N)]
    features = np.stack([arr for arr, ns in labels2D])
    counts = np.array([ns for arr, ns in labels2D])
    return features, counts
    
def analyse_sim(d,thresh=.7):
    d['t'] = t = np.array([save['sim_time'] for save in d['saves']])
    d['grid_old'] = x_old, z_old = flt.load_data(d['files'][0][0],'x/1.0','z/1.0',group='scales')
    d['x'] = x = np.linspace(-1,1,len(x_old),endpoint=False)
    d['z'] = z = np.linspace(-1,1,len(x_old),endpoint=False)
    d['grid_new'] = xx, zz = np.meshgrid(x,z,indexing='ij')
    measure = 4/(d['x'].size*d['z'].size)
    
    files = d['files']
    c1s_old, c2s_old = [np.vstack([flt.load_data(f[0],n,group='tasks',sel=np.s_[(f[0]!=files[0][0]):])[0] for f in files]) for n in ['c1','c2']]
    nt, nx, nz = len(t),len(x),len(z)
    d['c1'], d['c2'] = np.zeros((nt,nx,nz)), np.zeros((nt,nx,nz))
    for i in range(nt): d['c1'][i], d['c2'][i] = recentre_drop((x_old,z_old),c1s_old[i],c2s_old[i])
    d['c1b'], d['c2b'] = d['c1']>thresh, d['c2']>thresh
    d['c13D'], d['n13D'] = ndi.label(d['c1b'],structure=np.ones((3,3,3)))
    d['c13D'], d['n13D'] = ndi.label(d['c2b'],structure=np.ones((3,3,3)))
    d['c12D'], d['n12D'] = label_2D(d['c1b'])
    d['c22D'], d['n22D'] = label_2D(d['c2b'])
    d['c1_boxes'], d['c2_boxes'] = [[ndi.find_objects(d[f'{f}2D'][i]) for i in range(nt)] for f in ['c1','c2']]
    d['c1_sizes'], d['c2_sizes'] = [[[measure*ndi.sum_labels(d[f'c{k}b'][i],labels=d[f'c{k}2D'][i],index=j) for j in range(1,d[f'n{k}2D'][i]+1)] for i in range(nt)] for k in [1,2]]
    for k in [1,2]:
        d[f'c{k}_circ'] = [[circularity(d[f'c{k}b'][i],d[f'c{k}2D'][i],j+1,sel) for j, sel in enumerate(d[f'c{k}_boxes'][i])] for i in range(nt)]
    d['shape'] = [shape(d['c1'][i],d['c2'][i],d['n12D'][i],d['n22D'][i]) for i in range(nt)]
    return d

def circularity(binary,labels,label,sel):
    xmin, zmin = sel[0].start, sel[1].start
    x0, z0 = ndi.center_of_mass(binary[sel],labels=labels[sel],index=label)
    xi, zi = [np.arange(si) for si in binary[sel].shape]
    di = np.sqrt((xi-x0)[:,None]**2 + (zi-z0)[None,:]**2)
    A = (binary[sel]*(labels[sel]==label)).sum()
    r0 = np.sqrt(A/np.pi)
    inside = di < r0
    overlap = (binary[sel]*(labels[sel]==label)*inside).sum()/A
    linf_dist = int(max(np.abs(xi-round(x0)).max(), np.abs(zi-round(z0)).max()))
    square = np.zeros((2*linf_dist+1,2*linf_dist+1))
    big_sel = (slice(linf_dist-int(round(x0)),
                     linf_dist-int(round(x0))+xi.max()+1),
               slice(linf_dist-int(round(z0)),
                     linf_dist-int(round(z0))+zi.max()+1))
#     print(big_sel[0].stop-big_sel[0].start,big_sel[1].stop-big_sel[1].start)
#     print(sel[0].stop-sel[0].start,sel[1].stop-sel[1].start)
    square[big_sel] = binary[sel]*(labels[sel]==label)
#     big_sel = (slice(xmin+round(x0) - linf_dist,xmin+round(x0) + linf_dist),
#                slice(zmin+round(z0) - linf_dist,zmin+round(z0) + linf_dist))
    symmetry = (square*square[::-1].T).sum()/A
    return overlap, symmetry

def shape(c1,c2,n1,n2):
    if n1 > 1 or n2 > 1:
        return 'nonequilibrium'
    else:
        if n1 == 0:
            return 'c2 drop'
        elif n2 == 0:
            return 'c1 drop'
        elif ((c1>.3)&(1-c1-c2>.3)).sum() > 0:
            return 'crescent'
        elif c1.max() > .9 and c2.max() > .9:
            return 'shell'

def last_change(l):
    final = l[-1]
    for i in range(len(l)-1,-1,-1):
        if l[i] != final:
            return i+1
    return 0

# Analysis

In [ ]:
# sims = [0,1,2,4,5,6,7]
sims = list(range(42+7))
sim_dirs = {sim:{} for sim in sims}
for sim in sim_dirs:
    sim_dirs[sim] = {}
    d = sim_dirs[sim]
    d['series_name'] = 'chs-2D-comparison-T-1e-2' if sim < 42 else 'ch-2D-comparison-T-1e-2'
    d['save_series'] = [d["series_name"]+f'-{suff}' for suff in range(9)]
    d['index'] = sim if sim < 42 else sim - 42
    d['files'] = [glob.glob(f'hpc/conservation/data/{series}/analysis-{series}-{d["index"]:0>3d}/*.h5') for series in d['save_series']]
    d['files'] = [f for f in d['files'] if f]
    d['save_series'] = [s for i, s in enumerate(d['save_series']) if i < len(d['files'])]
    d['saves'] = list_saves(d['files'])
    if sim < 42: d['saves'] = filter_saves(d['saves'])
    d['sim_times'] = np.array([save['sim_time'] for save in d['saves']])
    d['parameters'] = pd.read_csv(f'hpc/conservation/parameters/parameters-{d["save_series"][0]}.csv',index_col=0).loc[d['index']]
    d = {**d, **analyse_sim(d)}
    print(sim)

In [ ]:
for sim in sim_dirs:
    d = sim_dirs[sim]
    times = np.array([save['sim_time'] for save in d['saves']])
    d['final_shape'] = d['shape'][-1]
    d['end_time'] = times[last_change(d['shape'])]
    d['max_n1'] = max(d['n12D'])
    d['max_n2'] = max(d['n22D'])
    d['t_steady_n1'] = times[last_change(d['n12D'])]
    d['t_steady_n2'] = times[last_change(d['n22D'])]    

In [ ]:
p0 = list(sim_dirs.values())[0]['parameters']
parameters = pd.DataFrame(columns = p0.keys())
for sim in sim_dirs:
    d = sim_dirs[sim]
    p = d['parameters']
    if sim >= 42: p = p.drop('initial_condition')
    parameters.loc[sim] = p.array

In [ ]:
for key in ['final_shape','end_time','max_n1','max_n2','t_steady_n1','t_steady_n2']:
    parameters[key] = [d[key] for sim, d in sim_dirs.items()]

# parameters.loc[[12],['final_shape','end_time']] = np.array([['crescent'], [200]]).T


In [ ]:
# from matplotlib import ticker

# fig, ax = plt.subplots(figsize=(4,3))
# for i, d in [(i, sim_dirs[i]) for i in [4,5,6]]:
#     ax.loglog(d['t'][2:],d['n12D'][2:],
#             label=f'CH $ε={parameters.loc[i]["ε"]}$')
# for i, d in [(i, sim_dirs[i]) for i in [0,1,2]]:
#     ax.loglog(d['t'][2:],d['n12D'][2:],
#             f'C{i}--',label=f'CHS $ε={parameters.loc[i]["ε"]}$')
# for i, ε in enumerate([1e-2,5e-3,2e-3]):
#     ax.loglog(d['t'],(1/6)*d['t']**(-1/2)/ε,f'C{i}',linestyle='dashdot')
# ax.loglog(d['t'],1e-5*d['t'],'k',linestyle='dashdot',label=r'$\frac{1}{6}~ε^{-1}t^{-1/2}$')
# ax.legend(bbox_to_anchor=(1.05,1),frameon=False,loc='upper left')
# ax.set(xlim=[3e-2,2e-1],ylim=[3e1,1e3],xlabel='Time $t$', ylabel='$c_1$ droplets',xticks=[])
# ticks=[3e-2,5e-2,1e-1,2e-1]
# ax.xaxis.set_minor_formatter(ticker.NullFormatter())
# plt.xticks(ticks, [ f"{x:.2f}" for x in ticks ])
# for spine in ['top','right']: ax.spines[spine].set_visible(False)
# plt.savefig('hpc/conservation/figures/final/droplet-number-eps.pdf',bbox_inches='tight')

# Plots

In [ ]:
groups = {(False,0,4):0,
          (True,'0',4):1,
          (True,'0',5):2,
          (True,'z',4):3,
          (True,'z',5):4,
          (True,'x',4):5,
          (True,'x',5):6,}
parameters['series'] = parameters[['flow','gravity','script']].apply(lambda row: groups[tuple(row)],axis=1)
series_labels = {
    0: 'CH',
    5: 'CHSB x',
    3: 'CHSB z',
    1: 'CHS',
    6: 'CHNSB x',
    4: 'CHNSB z',
    2: 'CHNS'}
marker_colors = {0:'gray',
                 1:'C3',
                 2:'maroon',
                 3:'C2',
                 4:'limegreen',
                 5:'C0',
                 6:'navy'}


marker_dics = lambda color, shape: {
    'crescent':{'marker':'^','facecolor':'none','edgecolor':color},
    'shell':{'marker':'o','facecolor':'none','edgecolor':color},
    'c2 drop':{'marker':'o','color':color},
}[shape]

from matplotlib.lines import Line2D

## Final shape and time to equilibrium

In [ ]:
plot_series = [0,1,3,5]
fig, ax = plt.subplots(figsize=(5,3))
for series in plot_series:
    subparams = parameters[parameters['series'] == series]
    final_shapes = subparams['final_shape']
    color = marker_colors[series]
    for sim in subparams.index:
        c10, end_time, final_shape = subparams.loc[sim,['c10','end_time','final_shape']]
        ax.scatter(float(end_time),float(c10),s=100,linewidth=3,**marker_dics(color,final_shape))
for spine in ['top','right']: ax.spines[spine].set_visible(False)        
ax.set(ylabel='Concentration ratio $c_1:,c_2$',
       xlabel='Time to equilibrium $t_{eq}$',xlim=[-10,300],
       title='Equilibrium shape and time vs concentrations')

legend_labels = ['CH','CHS','CHNS','CHSB z','CHNSB z','CHSB x','CHNSB x']
custom_lines = [Line2D([0], [0], color=marker_colors[s], lw=4, label=legend_labels[s]) for s in plot_series]
shape_lines = [Line2D([0], [0], marker='^', color='k', label='Crescent',markerfacecolor='none', markersize=9,linestyle='none',markeredgewidth=3),
          Line2D([0], [0], marker='o', color='k', label='Shell',markerfacecolor='none', markersize=11,linestyle='none',markeredgewidth=3),
          Line2D([0], [0], marker='o', color='k', label='$c_2$ drop',markerfacecolor='k', markersize=11,linestyle='none',markeredgewidth=3)]
ax.legend(handles=shape_lines+custom_lines,loc='center right',
          ncol=1,frameon=False,handlelength=1)

plt.savefig('hpc/conservation/figures/final/image-analysis/end_times.pdf',bbox_inches='tight')

## Droplet numbers over time

In [ ]:
p = parameters

plots = [((0,0),'1',.3),
         ((1,0),'1',.4),
         ((2,0),'1',.5),
         ((0,1),'2',.7),
         ((1,1),'2',.6),
         ((2,1),'2',.5)]

t0 = 10**np.linspace(-2,2)

In [ ]:
s = p['series']
c10s = p['c10']
fig, ax = plt.subplots(3,2,figsize=(5,4),sharex=True,sharey=True)
for pl in plots:
    axij, ni, cj = pl
    ylabel = f'$c_1:c_2$ = ${10*cj:.0f}:{10*(1-cj):.0f}$'
    for series in plot_series:
        sub_series = p[(s == series) & (c10s == cj)]
        for i, sim in enumerate(sub_series.index):
            d = sim_dirs[sim]
            ax[axij].loglog(d['t'],d[f'n{ni}2D'],label=series_labels[series],color=marker_colors[series])
    ax[axij].set(ylabel=ylabel)
#     ax[axij].loglog(t0,1e2*(t0/1e-2)**(-1/3),'k--',linewidth=1,zorder=-1)
    ax[axij].loglog(t0,1e2*(t0/1e-2)**(-1/2),'k--',linewidth=1,zorder=-1,label='$t^{-1/2}$')
    ax[axij].grid(True)
    for spine in ['top', 'right']:
        ax[axij].spines[spine].set_visible(False)    
ax[0,0].set_title('$c_1$ droplets',fontsize=14,pad=22)
ax[0,1].set_title('$c_2$ droplets',fontsize=14,pad=22)
fig.supylabel('Droplet count',x=-.05,fontsize=14)
fig.supxlabel('Time $t$',y=.0,fontsize=14)
ax[0,0].set(xlim=[1e-2,1.1e2],ylim=[1,1.05e2],xticks=10**np.arange(-2,2.01,1))
ax[0,0].legend(loc='upper center',frameon=False,bbox_to_anchor=(1.1,1.35),ncol=5,handlelength=1,columnspacing=1)
plt.savefig('hpc/conservation/figures/final/image-analysis/particle-number-vs-time.pdf',bbox_inches='tight')

## Analysis pipeline

In [ ]:
plt.pcolormesh(xx,zz,c1)
fig = plt.gcf()
ax = plt.gca()
ax.set(aspect=1)

In [ ]:
fig, ax = plt.subplots(1,4,figsize=(22*.4,(22/4)*.4))
ax = ax.flatten()

it = 32
save = d['saves'][it]
x0, z0 = flt.load_data(save['file'],'x/1.0','z/1.0',group='scales')
x1, z1 = d['x'],d['z']
c1,c2 = flt.load_data(save['file'],'c1','c2',group='tasks',sel=np.s_[save['save_index'],...])
c10 = uniform_interpolate_chebyshev(c1,(x0,z0),(x1,z1))
c20 = uniform_interpolate_chebyshev(c2,(x0,z0),(x1,z1))

# plot original data
plot_concentrations(c10,c20,ax[0])

# plot shifted data
plot_concentrations(d['c1'][it],d['c2'][it],ax[1])

# binary threshold
binary = d['c1b'][it]
ax[2].imshow(binary.T,cmap='Greys_r',interpolation='none')

# label features
labels = d['c12D'][it]
ax[3].imshow(labels.T,cmap='nipy_spectral',interpolation='none')

# # select one droplet
# label = 1
# bbox = d['c1_boxes'][it][label-1]
# measure = 4/(d['x'].size*d['z'].size)
# area = measure*ndi.sum_labels(d['c1b'][it],labels=d['c12D'][it],index=label)
# xmin, zmin = bbox[0].start, bbox[1].start
# x0, z0 = ndi.center_of_mass(binary[bbox],labels=labels[bbox],index=label)
# xi, zi = [np.arange(si) for si in binary[bbox].shape]
# di = np.sqrt((xi-x0)[:,None]**2 + (zi-z0)[None,:]**2)
# A = (binary[bbox]*(labels[bbox]==label)).sum()
# r0 = np.sqrt(A/np.pi)
# inside = di < r0
# overlap = (binary[bbox]*(labels[bbox]==label)*inside).sum()/A
# linf_dist = int(max(np.abs(xi-round(x0)).max(), np.abs(zi-round(z0)).max()))
# square = np.zeros((2*linf_dist+1,2*linf_dist+1))
# big_sel = (slice(linf_dist-int(round(x0)),
#                  linf_dist-int(round(x0))+xi.max()+1),
#            slice(linf_dist-int(round(z0)),
#                  linf_dist-int(round(z0))+zi.max()+1))
# # print(big_sel[0].stop-big_sel[0].start,big_sel[1].stop-big_sel[1].start)
# # print(bbox[0].stop-bbox[0].start,bbox[1].stop-bbox[1].start)
# square[big_sel] = binary[bbox]*(labels[bbox]==label)
# big_sel = (slice(xmin+round(x0) - linf_dist,xmin+round(x0) + linf_dist),
#            slice(zmin+round(z0) - linf_dist,zmin+round(z0) + linf_dist))
# symmetry = (square*square[::-1].T).sum()/A

# ax[4].imshow(labels[big_sel],cmap='nipy_spectral',vmin=[0,labels.max()],interpolation='none')

# ax[5].imshow(square,cmap='Greys_r',interpolation='none')
# ax[5].scatter([linf_dist],[linf_dist],c='k')

# circ0 = plt.Circle((linf_dist, linf_dist),(di*binary[bbox]).max(), color='C3', fill=False, linewidth=2)
# circ1 = plt.Circle((linf_dist, linf_dist),r0, color='C0', fill=False, linewidth=2)
# ax[6].imshow(square,cmap='Greys_r',interpolation='none')
# ax[6].scatter([linf_dist],[linf_dist],c='k')
# ax[6].add_patch(circ0)
# # ax[5].add_patch(circ1)

# intersections = np.zeros(square.shape)
# intersections[square==1] = 1
# intersections[square[::-1]==1] = 2
# intersections[(square==1) & (square[::-1]==1)] = 3
# ax[7].imshow(intersections,cmap='nipy_spectral',interpolation='none')
# ax[7].scatter([linf_dist],[linf_dist],c='k')
for axi in ax:
    axi.set(xticks=[],yticks=[])
ax[0].set_title('1. Uniform interpolation',fontsize=10)
ax[0].set_xlabel('RectBivariateSpline',fontsize=10)
ax[1].set_title('2. Recentering',fontsize=10)
ax[1].set_xlabel('skfmm.distance',fontsize=10)
ax[2].set_title('3. Binary threshold',fontsize=10)
ax[2].set_xlabel('c1 > 0.7',fontsize=10)
ax[3].set_title('4. Feature labelling',fontsize=10)
ax[3].set_xlabel('ndimage.label',fontsize=10)
fig.set_facecolor('w')
plt.savefig('hpc/conservation/figures/final/image-analysis/pipeline.pdf',bbox_inches='tight')

# 3D

In [ ]:
from scipy.interpolate import RegularGridInterpolator
def uniform_interpolate_3D(arr,grid_old,grid_new):
    return RegularGridInterpolator(grid_old, arr, method='linear')(grid_new)

def recentre_drop_3D(grids_old,grids_new,c1,c2):
    c3 = 1-c1-c2
    c1,c2,c3 = [uniform_interpolate_3D(arr,grids_old,grids_new) for arr in [c1,c2,c3]]
    dist = distance(c3-.5,dx=float(grids_new[0][1,0,0]-grids_new[0][0,0,0]),periodic=True)
    minimum_ind_x = np.unravel_index(np.argmin(dist,axis=None),dist.shape)[0]
    centre = len(grids_old[0])//2
    shift = centre - minimum_ind_x
    return np.roll(c1,shift,axis=0), np.roll(c2,shift,axis=0)

def label_3D(arr):
    N = len(arr)
    labels3D = [ndi.label(arr[i],structure=np.ones((3,3,3))) for i in range(N)]
    features = np.stack([arr for arr, ns in labels3D])
    counts = np.array([ns for arr, ns in labels3D])
    return features, counts
    
def analyse_sim(d,thresh=.7):
    d['t'] = t = np.array([save['sim_time'] for save in d['saves']])
    d['grid_old'] = x_old, y_old, z_old = flt.load_data(d['files'][0],'x/1.0','y/1.0','z/1.0',group='scales')
    d['x'] = x = x_old[1::2]#np.linspace(-1,1,len(x_old),endpoint=False)
    d['y'] = y = x_old[1::2]#np.linspace(-1,1,len(x_old),endpoint=False) # subsample!
    d['z'] = z = x_old[1::2]#np.linspace(-1,1,len(x_old),endpoint=False)
    d['grid_new'] = xx, yy, zz = np.meshgrid(x,y,z,indexing='ij')
    measure = 2**3/(d['x'].size*d['y'].size*d['z'].size)
    
    files = d['files']
    nt, nx, ny, nz = len(t),len(x),len(y),len(z)
    d['c1'], d['c2'] = np.zeros((nt,nx,ny,nz)), np.zeros((nt,nx,ny,nz))
    for i in range(nt): 
        save = d['saves'][i]
        c1s_old, c2s_old = flt.load_data(save['file'],'c1','c2',group='tasks',sel=np.s_[save['save_index'],...])
        d['c1'][i], d['c2'][i] = recentre_drop_3D((x_old,y_old,z_old),(xx,yy,zz),c1s_old,c2s_old)
    d['c1b'], d['c2b'] = d['c1']>thresh, d['c2']>thresh
    d['c14D'], d['n14D'] = ndi.label(d['c1b'],structure=np.ones((3,3,3,3)))
    d['c14D'], d['n14D'] = ndi.label(d['c2b'],structure=np.ones((3,3,3,3)))
    d['c13D'], d['n13D'] = label_3D(d['c1b'])
    d['c23D'], d['n23D'] = label_3D(d['c2b'])
    d['c1_boxes'], d['c2_boxes'] = [[ndi.find_objects(d[f'{f}3D'][i]) for i in range(nt)] for f in ['c1','c2']]
    d['c1_sizes'], d['c2_sizes'] = [[[measure*ndi.sum_labels(d[f'c{k}b'][i],labels=d[f'c{k}3D'][i],index=j) for j in range(1,d[f'n{k}3D'][i]+1)] for i in range(nt)] for k in [1,2]]
#     for k in [1,2]:
#         d[f'c{k}_circ'] = [[circularity(d[f'c{k}b'][i],d[f'c{k}2D'][i],j+1,sel) for j, sel in enumerate(d[f'c{k}_boxes'][i])] for i in range(nt)]
#     d['shape'] = [shape(d['c1'][i],d['c2'][i],d['n12D'][i],d['n22D'][i]) for i in range(nt)]
    return d

In [ ]:
# load data
save_series = ['chs-3D-comparison-SBDF2']
sims = [0]
parameters = pd.read_csv(f'hpc/conservation/parameters/parameters-{save_series[0]}.csv',index_col=0)
test = 0
sim_dirs = {sim:{} for sim in sims[:test]}
for sim in sims:
    sim_dirs[sim] = load_saves(sim)
    sim_dirs[sim] = analyse_sim(sim_dirs[sim])

In [ ]:
sim = 0
d = sim_dirs[sim]
it = -1
save = d['saves'][it]

In [ ]:
from matplotlib import cm

In [ ]:
yy,zz = np.meshgrid(d['y'],d['z'],indexing='ij')

In [ ]:
for i in range(0,96,8):
    print(cm.viridis(i/96))
    plt.contour(*np.meshgrid(d['y'],d['z'],indexing='ij'),d['c13D'][-1][i],[0.5],colors=[cm.viridis(i/200),cm.viridis((i+100)/200)])

In [ ]:
for i in range(0,96,8):
    print(cm.viridis(i/96))
    plt.contour(*np.meshgrid(d['y'],d['z'],indexing='ij'),d['c1'][-1][i],[.5],colors=[cm.viridis(i/96)])

In [ ]:
def uniform_interpolate_3D(arr,grid_old,grid_new):
    return RegularGridInterpolator(grid_old, arr, method='linear')(tuple(grid_new))